In [8]:
import pickle
import numpy as np
import mne

from scipy import signal
from scipy.stats import zscore
from scipy.interpolate import interp1d

# Load the pickle file
with open('practice.p', 'rb') as file:
    data = pickle.load(file)

# Inspect the top-level keys in the data
print("Keys in the data:", data.keys())

Keys in the data: dict_keys(['DSI24', 'illumiReadSwypeEventMarkerLSL', 'illumiReadSwypeUserInputLSL', 'VarjoEyeTrackingLSL'])


In [9]:
# Inspect DSI24 (EEG data)
eeg_data = data['DSI24']
print("EEG Data Structure:", type(eeg_data))
print("EEG Data Array Shape:", np.array(eeg_data[0]).shape)  # EEG signals
print("EEG Timestamps Shape:", len(eeg_data[1]))  # EEG timestamps

# Inspect illumiReadSwypeEventMarkerLSL (stimulus markers)
event_markers = data['illumiReadSwypeEventMarkerLSL']
print("Event Markers Data Structure:", type(event_markers))
print("Event Marker Values Shape:", np.array(event_markers[0]).shape)
print("Event Marker Timestamps Shape:", len(event_markers[1]))

# Inspect illumiReadSwypeUserInputLSL (selection events)
user_input = data['illumiReadSwypeUserInputLSL']
print("User Input Data Structure:", type(user_input))
print("User Input Array Shape:", np.array(user_input[0]).shape)
print("User Input Timestamps Shape:", len(user_input[1]))

# Inspect VarjoEyeTrackingLSL (eye-tracking data)
eye_tracking = data['VarjoEyeTrackingLSL']
print("Eye Tracking Data Structure:", type(eye_tracking))
print("Eye Tracking Array Shape:", np.array(eye_tracking[0]).shape)
print("Eye Tracking Timestamps Shape:", len(eye_tracking[1]))

EEG Data Structure: <class 'list'>
EEG Data Array Shape: (24, 71250)
EEG Timestamps Shape: 71250
Event Markers Data Structure: <class 'list'>
Event Marker Values Shape: (3, 20)
Event Marker Timestamps Shape: 20
User Input Data Structure: <class 'list'>
User Input Array Shape: (11, 15844)
User Input Timestamps Shape: 15844
Eye Tracking Data Structure: <class 'list'>
Eye Tracking Array Shape: (39, 38891)
Eye Tracking Timestamps Shape: 38891


In [10]:
# Align timestamps for synchronization
eeg_timestamps = np.array(eeg_data[1])  # EEG timestamps
event_timestamps = np.array(event_markers[1])  # Event marker timestamps
selection_timestamps = np.array(user_input[1])  # User input timestamps
eye_tracking_timestamps = np.array(eye_tracking[1])  # Eye tracking timestamps

# Print the time range for each dataset
print("EEG Time Range:", eeg_timestamps.min(), "to", eeg_timestamps.max())
print("Event Markers Time Range:", event_timestamps.min(), "to", event_timestamps.max())
print("User Input Time Range:", selection_timestamps.min(), "to", selection_timestamps.max())
print("Eye Tracking Time Range:", eye_tracking_timestamps.min(), "to", eye_tracking_timestamps.max())

# Check for time overlap
overlap_start = max(eeg_timestamps.min(), event_timestamps.min(), selection_timestamps.min(), eye_tracking_timestamps.min())
overlap_end = min(eeg_timestamps.max(), event_timestamps.max(), selection_timestamps.max(), eye_tracking_timestamps.max())
print("Overlap Time Range:", overlap_start, "to", overlap_end)

EEG Time Range: 61163.35202906667 to 61417.53702906667
Event Markers Time Range: 61168.0556045 to 61412.5162805
User Input Time Range: 61215.1974676 to 61412.5055186
Eye Tracking Time Range: 61214.698143015004 to 61412.495215111
Overlap Time Range: 61215.1974676 to 61412.495215111


In [11]:
# Convert EEG data (list of lists/arrays) to a NumPy array
eeg_data_array = np.array(eeg_data[0], dtype=float)  # EEG signals as a 2D array (channels × samples)
eeg_timestamps_array = np.array(eeg_timestamps, dtype=float)  # Timestamps as a 1D array

# Find EEG indices that are within the overlapping time range
eeg_indices = np.where((eeg_timestamps_array >= overlap_start) & (eeg_timestamps_array <= overlap_end))[0]

# Extract overlapping EEG data and timestamps
overlapping_eeg = eeg_data_array[:, eeg_indices]  # Use advanced indexing for NumPy arrays
overlapping_timestamps = eeg_timestamps_array[eeg_indices]  # Corresponding timestamps

print("Overlapping EEG Shape:", overlapping_eeg.shape)
print("Overlapping EEG Timestamps Shape:", overlapping_timestamps.shape)

Overlapping EEG Shape: (24, 54192)
Overlapping EEG Timestamps Shape: (54192,)


In [12]:
def apply_filters(eeg_data, sfreq, lowcut, highcut, notch_freq, timestamps):
    """Same filtering but preserves timestamps"""
    # Design Butterworth bandpass filter
    order = 4
    nyquist = sfreq / 2
    b, a = signal.butter(order, [lowcut/nyquist, highcut/nyquist], btype='band')

    # Apply bandpass filter
    filtered_data = signal.filtfilt(b, a, eeg_data, axis=1)

    # Design and apply notch filter
    notch_b, notch_a = signal.iirnotch(notch_freq, Q=30, fs=sfreq)
    filtered_data = signal.filtfilt(notch_b, notch_a, filtered_data, axis=1)

    return filtered_data, timestamps

def detect_bad_channels(eeg_data, timestamps, z_threshold=3):
    """Same detection but includes timestamps"""
    channel_std = np.std(eeg_data, axis=1)
    channel_range = np.ptp(eeg_data, axis=1)

    std_z = zscore(channel_std)
    range_z = zscore(channel_range)

    bad_channels = np.where((np.abs(std_z) > z_threshold) |
                          (np.abs(range_z) > z_threshold))[0]

    return bad_channels, timestamps

def remove_artifacts(eeg_data, timestamps, threshold_std=5):
    """Same artifact removal but uses timestamps for interpolation"""
    data_clean = eeg_data.copy()

    chan_std = np.std(data_clean, axis=1, keepdims=True)
    chan_mean = np.mean(data_clean, axis=1, keepdims=True)

    artifact_mask = np.abs(data_clean - chan_mean) > (threshold_std * chan_std)

    for chan in range(data_clean.shape[0]):
        bad_samples = artifact_mask[chan, :]
        good_samples = ~bad_samples
        if np.any(bad_samples):
            interp_func = interp1d(
                timestamps[good_samples],
                data_clean[chan, good_samples],
                kind='linear',
                bounds_error=False,
                fill_value='extrapolate'
            )
            data_clean[chan, bad_samples] = interp_func(timestamps[bad_samples])

    return data_clean, timestamps

def calculate_signal_quality(eeg_data, timestamps):
    """Same quality metrics but includes timestamps"""
    signal_power = np.mean(np.square(eeg_data), axis=1)
    noise_power = np.var(np.diff(eeg_data, axis=1), axis=1)

    epsilon = 1e-10
    noise_power = noise_power + epsilon

    snr = np.zeros_like(signal_power)
    valid_idx = (signal_power > 0) & (noise_power > 0)
    snr[valid_idx] = 10 * np.log10(signal_power[valid_idx] / noise_power[valid_idx])

    rms = np.sqrt(np.mean(np.square(eeg_data), axis=1))

    metrics = {
        'SNR': snr,
        'RMS': rms,
        'Signal_Power': signal_power,
        'Noise_Power': noise_power,
        'Invalid_Channels': np.where(~valid_idx)[0],
        'Time_Range': timestamps[-1] - timestamps[0]  # Added time range info
    }

    if len(metrics['Invalid_Channels']) > 0:
        print(f"\nWarning: Found {len(metrics['Invalid_Channels'])} channels with invalid SNR values")
        print(f"Problematic channels: {metrics['Invalid_Channels']}")
        print("These channels might need additional cleaning or inspection")

    return metrics

In [13]:
# Calculate sampling rate
sampling_rate = len(overlapping_eeg[0]) / (overlapping_timestamps[-1] - overlapping_timestamps[0])
nyquist = sampling_rate / 2
print(f"Sampling rate: {sampling_rate:.2f} Hz")

# Define filter parameters
lowcut = 0.5
highcut = 50
notch_freq = 60

# Apply filters
filtered_eeg, filtered_timestamps = apply_filters(overlapping_eeg, sampling_rate, lowcut, highcut, notch_freq, overlapping_timestamps)

# Detect bad channels
bad_channels, _ = detect_bad_channels(filtered_eeg, filtered_timestamps)
print(f"Detected bad channels: {bad_channels}")

# Remove artifacts
clean_eeg, clean_timestamps = remove_artifacts(filtered_eeg, filtered_timestamps)

# Calculate metrics
quality_metrics = calculate_signal_quality(clean_eeg, clean_timestamps)

# Print summary statistics
print("\nSignal Quality Metrics:")
print(f"Time range preserved: {quality_metrics['Time_Range']:.2f} seconds")
print(f"Average SNR across valid channels: {np.mean(quality_metrics['SNR'][quality_metrics['SNR'] != 0]):.2f} dB")
print(f"Average RMS amplitude: {np.mean(quality_metrics['RMS']):.2f} µV")
print(f"Signal Power range: {np.min(quality_metrics['Signal_Power']):.2e} to {np.max(quality_metrics['Signal_Power']):.2e}")
print(f"Noise Power range: {np.min(quality_metrics['Noise_Power']):.2e} to {np.max(quality_metrics['Noise_Power']):.2e}")

# Channel-specific analysis
for i in range(len(quality_metrics['SNR'])):
    if quality_metrics['SNR'][i] != 0:
        print(f"\nChannel {i+1}:")
        print(f"  SNR: {quality_metrics['SNR'][i]:.2f} dB")
        print(f"  RMS: {quality_metrics['RMS'][i]:.2f} µV")

Sampling rate: 274.67 Hz
Detected bad channels: []

Problematic channels: [20]
These channels might need additional cleaning or inspection

Signal Quality Metrics:
Time range preserved: 197.30 seconds
Average SNR across valid channels: 18.63 dB
Average RMS amplitude: 115.27 µV
Signal Power range: 0.00e+00 to 8.42e+04
Noise Power range: 1.00e-10 to 4.36e+02

Channel 1:
  SNR: 17.56 dB
  RMS: 108.34 µV

Channel 2:
  SNR: 16.15 dB
  RMS: 53.00 µV

Channel 3:
  SNR: 19.82 dB
  RMS: 178.56 µV

Channel 4:
  SNR: 23.96 dB
  RMS: 180.64 µV

Channel 5:
  SNR: 19.02 dB
  RMS: 68.66 µV

Channel 6:
  SNR: 21.22 dB
  RMS: 126.40 µV

Channel 7:
  SNR: 22.11 dB
  RMS: 153.96 µV

Channel 8:
  SNR: 23.17 dB
  RMS: 86.30 µV

Channel 9:
  SNR: 20.76 dB
  RMS: 80.39 µV

Channel 10:
  SNR: 23.86 dB
  RMS: 163.29 µV

Channel 11:
  SNR: 14.04 dB
  RMS: 49.64 µV

Channel 12:
  SNR: 17.79 dB
  RMS: 73.23 µV

Channel 13:
  SNR: 22.22 dB
  RMS: 142.05 µV

Channel 14:
  SNR: 24.32 dB
  RMS: 164.13 µV

Channel 15:

In [14]:
import pandas as pd
import json


# Enhanced save function with more comprehensive metadata
def save_processed_eeg_enhanced(eeg_data, timestamps, filename):
    # Create DataFrame with timestamps
    data_dict = {'Timestamp': timestamps}

    # Add channel data with quality indicators and interpolation status
    for i in range(eeg_data.shape[0]):
        channel_name = f'Channel_{i+1}'
        if i in bad_channels:
            channel_name += '_interpolated'
        data_dict[channel_name] = eeg_data[i, :]

    df = pd.DataFrame(data_dict)

    # Save data to CSV
    csv_filename = f"{filename}.csv"
    df.to_csv(csv_filename, index=False)

    print(f"Data saved to {csv_filename}")

    return df

save_processed_eeg_enhanced(clean_eeg, clean_timestamps, "cleaned_eeg_data")

Data saved to cleaned_eeg_data.csv


,Timestamp,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_15,Channel_16,Channel_17,Channel_18,Channel_19,Channel_20,Channel_21,Channel_22,Channel_23,Channel_24
0,61215.197585,-0.000656,-0.000899,-8.041068,-0.000292,-0.002567,-0.001797,-0.004917,0.002202,0.001178,...,0.045235,0.009640,-0.001454,0.004887,0.003217,0.000283,0.0,-0.068090,-0.137459,-0.207612
1,61215.200918,-0.000635,-0.000877,-26.031161,-0.000045,-0.002524,-0.001719,-0.004966,0.002167,0.001179,...,0.046480,0.010293,-0.001278,0.004697,0.003128,0.000279,0.0,0.054447,-0.356274,-0.044487
2,61215.204251,-0.000613,-0.000855,-61.350246,0.000202,-0.002480,-0.001641,-0.005014,0.002131,0.001179,...,0.047720,0.010944,-0.001102,0.004506,0.003038,0.000275,0.0,0.097343,-0.447097,0.025670
3,61215.207585,-0.000590,-0.000832,-105.558223,0.000455,-0.002435,-0.001560,-0.005063,0.002094,0.001179,...,0.048983,0.011610,-0.000921,0.004310,0.002946,0.000271,0.0,0.052113,-0.395176,-0.003606
4,61215.210918,-0.000567,-0.000809,-129.261696,0.000714,-0.002388,-0.001478,-0.005112,0.002057,0.001180,...,0.050267,0.012290,-0.000737,0.004109,0.002851,0.000266,0.0,-0.021422,-0.306945,-0.055442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54187,61412.480918,71.855182,9.101630,-1.905722,-12.467333,-87.576242,-14.456167,16.877790,15.496784,-18.105357,...,-6.612259,13.409222,28.716738,-9.866168,-47.267876,5.259978,0.0,0.426123,0.210139,-0.025385
54188,61412.484251,101.226060,18.817411,10.412924,-3.282155,-73.845764,4.853901,49.327937,21.042239,-10.284187,...,8.407107,30.455416,40.646625,-1.730396,-35.595621,5.961451,0.0,0.391161,0.335185,0.043381
54189,61412.487585,111.611772,15.366422,10.579166,0.624548,-76.672153,21.970027,61.486926,15.718234,-10.171323,...,10.218605,33.782213,33.737081,1.134676,-33.420269,5.383325,0.0,0.290169,0.407474,0.084188
54190,61412.490918,84.796344,5.989761,2.138966,0.976709,-80.950502,22.050302,47.922546,7.153477,-14.228533,...,1.734016,24.177597,13.994044,-1.151547,-34.908086,3.539768,0.0,0.162502,0.299142,0.093218
